In [ ]:
! pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# 크롬 옵션 설정 (헤드리스 모드)
chrome_options = Options()
chrome_options.add_argument('--headless')  # GUI를 표시하지 않음
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
from datetime import datetime

include = input('기사 제목에 포함시킬 내용을 입력하세요: ')

while True:
    start_date = input('시작 날짜를 입력하세요(예시: 20xx-xx-xx): ')
    end_date = input('종료 날짜를 입력하세요(예시: 20xx-xx-xx): ')
    
    try:
        start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
        end_date_obj = datetime.strptime(end_date, "%Y-%m-%d")
        
        if end_date_obj >= start_date_obj:
            break
        else:
            print("종료 날짜가 시작 날짜보다 빠를 수 없습니다. 다시 입력해주세요.")
    
    except ValueError:
        print("날짜 형식이 올바르지 않습니다. 다시 입력해주세요.")


start_date = list(map(int, start_date.split('-')))
end_date = list(map(int, end_date.split('-')))

In [ ]:
def select_date(start_date, end_date):
    global driver
    time.sleep(3)

    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{start_date[0]-1989}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[2]/div/div/div/ul/li[{start_date[1]}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[3]/div/div/div/ul/li[{start_date[2]}]/a').click()

    # 종료 날짜 클릭하기
    driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[1]/span[3]/a').click()

    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{end_date[0]-1989}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[2]/div/div/div/ul/li[{end_date[1]}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[3]/div/div/div/ul/li[{end_date[2]}]/a').click()

    # 적용 클릭
    driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[3]/button').click()

In [ ]:
# 크롬 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

driver.get(url=f'https://search.naver.com/search.naver?where=news&query={include}')

# 최신순
driver.find_element(By.XPATH, '//*[@id="snb"]/div[1]/div/div[1]/a[2]').click()
# 옵션 클릭
driver.find_element(By.XPATH, '//*[@id="snb"]/div[1]/div/div[2]/a').click()
driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]').click()

select_date(start_date, end_date)

# 스크롤링을 위한 반복문
last_height = driver.execute_script("return document.body.scrollHeight")  # 현재 페이지의 전체 높이

while True:
    # 스크롤을 최하단으로 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 로드 시간 대기

    # 새로운 페이지 높이를 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")

    # 새로운 페이지 높이가 기존 높이와 같으면, 더 이상 스크롤 불가능
    if new_height == last_height:
        break
    last_height = new_height  # 마지막 높이 갱신

press = [element.get_attribute("textContent") for element in driver.find_elements(By.CLASS_NAME, "info.press")] # 언론사
title = [element.get_attribute("title") for element in driver.find_elements(By.CLASS_NAME, "news_tit")] # 기사 제목
link = [element.get_attribute("href") for element in driver.find_elements(By.CLASS_NAME, "news_tit")]   # 기사 링크

driver.quit()

for t, p, l in zip(title, press, link):
    print(f"언론사: {p} | 제목: {t} | 링크: {l}")

In [ ]:
import pandas as pd

# 데이터프레임 생성
df = pd.DataFrame({
    "언론사": press,
    "기사 제목": title,
    "링크": link
}).drop_duplicates(subset=['언론사','기사 제목']).reset_index(drop=True)
df.index = range(1, len(df) + 1)
df.rename_axis('순번', axis='index', inplace=True)
df.to_csv('article_dataframe.csv', index=False, encoding='utf-8-sig')

# 데이터프레임 출력
display(df)

In [ ]:
from google.colab import files

# CSV 파일 다운로드
files.download('article_dataframe.csv')